![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/annotation/text/italian/date_matcher_multi_language_it.ipynb)

# DateMatcher multi-language (Italian)
This annotator allows you to specify a source language that will be used to identify temporal keywords and extract dates.

In [ ]:
# Only run this cell when you are using Spark NLP on Google Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [ ]:
# Import Spark NLP
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

# Start Spark Session with Spark NLP
# start() functions has two parameters: gpu and spark23
# sparknlp.start(gpu=True) will start the session with GPU support
# sparknlp.start(spark23=True) is when you have Apache Spark 2.3.x installed
spark = sparknlp.start()

In [ ]:
spark

In [ ]:
sparknlp.version()

'4.3.1'

# Italian examples

### Let's import some articoles sentences from the news where relative dates are present.

In [ ]:
it_articles = [
  ("Così il ct azzurro Roberto Mancini, poco prima di entrare al Quirinale dove l'Italia campione d'Europa sta per essere accolta dal Presidente della Repubblica Sergio Mattarella oggi.",),
  ("I giocatori della nazionale italiana campione d'Europa sono stati ricevuti al Quirinale il 13 Luglio 2021 per un incontro con il presidente della Repubblica, Sergio Mattarella.",),
  ("Il presidente della Repubblica Sergio Mattarella ha ricevuto ieri, alle ore 17.00 al Quirinale, la Nazionale italiana di calcio vincitrice del Campionato Europeo UEFA Euro 2020 e Matteo Berrettini, finalista al Torneo di Wimbledon.",)
]

### Let's  fill a DataFrame with the text column

In [ ]:
articles_cols = ["text"]

df = spark.createDataFrame(data=it_articles, schema=articles_cols)

df.printSchema()
df.show()

root
 |-- text: string (nullable = true)

+--------------------+
|                text|
+--------------------+
|Così il ct azzurr...|
|I giocatori della...|
|Il presidente del...|
+--------------------+



### Now, let's create a simple pipeline to apply the DateMatcher, specifying the source language

In [ ]:
document_assembler = DocumentAssembler() \
            .setInputCol("text") \
            .setOutputCol("document")

date_matcher = DateMatcher() \
            .setInputCols(['document']) \
            .setOutputCol("date") \
            .setOutputFormat("MM/dd/yyyy") \
            .setSourceLanguage("it")

### Let's transform the DataFrame content to extract the dates

In [ ]:
assembled = document_assembler.transform(df)
date_matcher.transform(assembled).select('date').show(10, False)

+---------------------------------------------------+
|date                                               |
+---------------------------------------------------+
|[{date, 175, 183, 02/20/2023, {sentence -> 0}, []}]|
|[{date, 91, 102, 07/13/2021, {sentence -> 0}, []}] |
|[{date, 61, 69, 02/19/2023, {sentence -> 0}, []}]  |
+---------------------------------------------------+

